## Лекция 3  NER

### __Задача 1__:

Реализуйте 2 функции препроцессинга:

- Удалить именованные сущности с помощью natasha (https://github.com/natasha/yargy)
- Удалить именованные сущности с помощью deepmipt (https://github.com/deepmipt/ner)

# Наташа 

In [ ]:
!pip install deeppavlov

In [ ]:
!pip install natasha

In [ ]:
!python -m deeppavlov install ner_ontonotes_bert

In [10]:
#natasha part
from natasha import (
    Segmenter,
    NewsNERTagger,
    NewsEmbedding,
    Doc
)

emb = NewsEmbedding()
ner_tagger = NewsNERTagger(emb)
segmenter = Segmenter()

#deepmit part
from deeppavlov import configs, build_model
ner_model = build_model(configs.ner.ner_rus_bert, download=True)

2020-10-22 08:46:27.208 INFO in 'deeppavlov.download'['download'] at line 132: Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v1.tar.gz download because of matching hashes
2020-10-22 08:46:30.922 INFO in 'deeppavlov.download'['download'] at line 132: Skipped http://files.deeppavlov.ai/deeppavlov_data/ner_rus_bert_v1.tar.gz download because of matching hashes
2020-10-22 08:46:31.314 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_rus_bert/tag.dict]
2020-10-22 08:47:02.567 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/.deeppavlov/models/ner_rus_bert/model]


INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/ner_rus_bert/model


In [7]:
text = 'Посол Израиля на Украине Йоэль Лион признался, что пришел в шок, узнав о решении властей Львовской области объявить 2019 год годом лидера запрещенной в России Организации украинских националистов (ОУН) Степана Бандеры...'

In [76]:
# need to remove PERSON, LOCATION, ORGANISATION, TIME
def preprocess_with_natasha(text: str) -> str:
    text = str(text)
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_ner(ner_tagger)
    positions = []
    for element in doc.spans:
        positions.append([element.start, element.stop])
    lst = list(text)
    for i in positions[::-1]: # iterate from behind so index does not shrink inwards
        del lst[slice(*i)]
    text = ''.join(lst)
    text = text.replace('  ', ' ')
    text = text.replace('  ', ' ')
    text = text.replace(' .', '.')
    print(text)
    return text

In [12]:
preprocess_with_natasha(text)

Посол на признался, что пришел в шок, узнав о решении властей объявить 2019 год годом лидера запрещенной в...


'Посол на признался, что пришел в шок, узнав о решении властей объявить 2019 год годом лидера запрещенной в...'

# deepmipt

In [77]:
def preprocess_with_deepmipt(text: str) -> str:
  text = str(text)
  text = ner_model([text])
  # слова
  sentence = text[0][0]
  # теги к словам
  tags = text[1][0]
  # совмещаем в один объект
  tags_and_words = list(zip(sentence, tags))
  # сюда запишу где именованные сущности
  counter = []
  for entity in tags_and_words:
    if entity[1] != 'O':
      counter.append(tags_and_words.index(entity))
  no_named = [ elem for elem in tags_and_words if tags_and_words.index(elem) not in counter]
  # сюда запишу список нужных слов для возвращения к тексту от списка слов
  words = []
  for pair in no_named:
    words.append(pair[0])
  return ' '.join(words)

In [65]:
preprocess_with_deepmipt(text)

'Посол на признался , что пришел в шок , узнав о решении властей объявить 2019 год годом лидера запрещенной в . . .'

### Добавления к препроцессингу
Кроме того, для соответствия препроцессингу в предыдущем задании и в надежде улучшить результат я добавлю несколько операций над текстом в препроцессинге.


In [66]:
# для работы моего препроцессинга и запуска в дальнейшем с тф-идф и бм25
import pandas as pd
import numpy as np

import string # для .punctuatuion 
import math # bm25
from collections import OrderedDict # для сортировки
from collections import Counter # для матрицы бм25

from razdel import tokenize
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [67]:
morph = MorphAnalyzer()
stop = set(stopwords.words('russian'))

def prep(unprep):
    unprep = str(unprep)
    unprep = unprep.lower()
    unprep = unprep.replace('\n', ' ') #.punctuation не прочтет
    unprep = unprep.translate(str.maketrans('', '', string.punctuation))# https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string
    unprep_tokenized = list(tokenize(unprep))
    lemmas = [morph.parse(i.text)[0].normal_form for i in unprep_tokenized]
    words = [i for i in lemmas if i not in stop]
    return ' '.join(words)

### __Задача 2__:    
На предыдущем занятии вы реализовывали функции поиска ближайших ответов на запросы через TF-IDF и BM25. 
Сравните качество нахождения верного ответа для обоих методов в трех случаях:
- с функцией ```preprocess_with_natasha```
- с функцией ```preprocess_with_deepmipt```
- без препроцессинга

Для измерения качества используйте метрику accuracy. Считаем, что ответ верный, если он входит в топ-1.

## Что теперь делать?
Препроцессинг получился: работает наташа, диппавлов и мой небольшой препроцессинг из прошлой работы. Остается только скопировать предыдущую тетрадку и добавить к строчкам применения моей функции препроцессинга полученные новые: наташу и диппавлова.

Добавлять строчки придется с небольшими изменениями, потому что нужно посчитать точность. Кроме того, однокурсники посоветовали мне пользоваться tqdm, чтобы следить за реализацией препроцессинга, который может затягиваться.

Таким образом мне необходимо:
1. вставить базовый код прошлого задания
2. разобраться с tqdm 
3. запустить препроцессинг на тест и на трейн с tqdm
4. получить тест и трейн матрицы
5. посчитать точность выбора по связкам

In [69]:
# теперь читать можно сразу из экселя
answers_df = pd.read_excel("answers_base.xlsx")
questions_df = pd.read_excel("queries_base.xlsx")

In [70]:
# названия столбцов неудобно вызывать в pandas, переименую
# тематика не понадобится
answers_df.rename(columns={'Номер связки': 'connection', 'Текст вопросов': 'text'}, inplace=True)
questions_df.rename(columns={'Номер связки\n': 'connection', 'Текст вопроса': 'text'}, inplace=True)

In [ ]:
# делим на тест и трейн красиво, не слайсом
train_df, test_df = train_test_split(questions_df, test_size=0.3)
# объединяем все тренировочные тексты вопросов
qa_df = pd.concat([answers_df, train_df])

In [ ]:
qa_df = qa_df.drop(columns = ['Unnamed: 3','Unnamed: 4'])
qa_df

## НАТАША + TF-IDF


In [ ]:
# попробую сделать заполняемые строки tqdm
# ПРЕПРОЦЕССИНГ
from tqdm import tqdm
test_df_done = []
qa_df_done = []

for entry in tqdm(test_df['text']):
    test_df_done.append(prep(preprocess_with_natasha(entry)))
for entry in tqdm(qa_df['text']):
    qa_df_done.append(prep(preprocess_with_natasha(entry)))

both_done = qa_df_done + test_df_done

In [83]:
# сделаем фит и матрицы по нему для теста и трейна
vectorizer = TfidfVectorizer(both_done)
fit_vec = vectorizer.fit_transform(both_done)
X_train = vectorizer.transform(qa_df_done)
print(X_train.shape)
X_test = vectorizer.transform(test_df_done)
print(X_test.shape)

(1652, 7588)
(690, 7588)


In [93]:
# осталось только посчитать точность работы алгоритма

rating = X_train.dot(X_test.T).argmax(axis=0)
# в следующей ячейке у меня не получилось сделать проверку на nan через pandas,
# поэтому будет массив нампай
rating = np.array(rating)[0]
rating_en = enumerate(rating)

In [95]:
score = 0
# сравним связки
for test_index, prediction in rating_en:
    # проверка на NaN
    if math.isnan(test_df.iloc[test_index].connection) or math.isnan(qa_df.iloc[prediction].connection):
        continue

    # если в тесте и трейне совпала связка по посчитанным индексам из рейтинга, модели плюс очко
    if int(test_df.iloc[test_index].connection) == int(qa_df.iloc[prediction].connection):
        score += 1

In [96]:
"natasha + TF-IDF: " + str(score / len(rating))

'natasha + TF-IDF: 0.5246376811594203'

# DEEP PAVLOV + TF-IDF

на этом этапе оказалось, что нужно резать вхождения в диппавлов до 300 символов, которые примерно равны 512 токенам БЕРТа. Иначе ничего не будет работать. https://github.com/deepmipt/DeepPavlov/issues/839

Поэтому была переписана функция процессинга с диппавловым

In [98]:
def deepmipt_helper(text):
    text = ner_model([text])
  # слова
    sentence = text[0][0]
  # теги к словам
    tags = text[1][0]
  # совмещаем в один объект
    tags_and_words = list(zip(sentence, tags))
  # сюда запишу где именованные сущности
  counter = []
    for entity in tags_and_words:
        if entity[1] != 'O':
            counter.append(tags_and_words.index(entity))
    no_named = [ elem for elem in tags_and_words if tags_and_words.index(elem) not in counter]
  # сюда запишу список нужных слов для возвращения к тексту от списка слов
    words = []
    for pair in no_named:
        words.append(pair[0])
    return ' '.join(words)

def preprocess_with_deepmipt(text: str) -> str:
    list_text = []
    text = str(text)
  # к сожалению, боюсь не успеть написать красиво через sentensize. прошу, простите
    if len(text) > 300:
        list_text = [text[i: i + 300] for i in range(0, len(text), 300)]
        for part in list_text:
            part = deepmipt_helper(part)
        text = ' '.join(list_text)
    else:
        text = deepmipt_helper(text)
    return text

In [99]:
test_df_done = []
qa_df_done = []

for entry in tqdm(test_df['text']):
    test_df_done.append(prep(preprocess_with_deepmipt(entry)))
for entry in tqdm(qa_df['text']):
    qa_df_done.append(prep(preprocess_with_deepmipt(entry)))

both_done = qa_df_done + test_df_done

100%|██████████| 1652/1652 [16:09<00:00,  1.70it/s]


In [100]:
vectorizer = TfidfVectorizer(both_done)
fit_vec = vectorizer.fit_transform(both_done)
X_train = vectorizer.transform(qa_df_done)
print(X_train.shape)
X_test = vectorizer.transform(test_df_done)
print(X_test.shape)

(1652, 9775)
(690, 9775)


In [101]:
# осталось только посчитать точность работы алгоритма

rating = X_train.dot(X_test.T).argmax(axis=0)
# в следующей ячейке у меня не получилось сделать проверку на nan через pandas,
# поэтому будет массив нампай
rating = np.array(rating)[0]
rating_en = enumerate(rating)

In [102]:
score = 0
# сравним связки
for test_index, prediction in rating_en:
    # проверка на NaN
    if math.isnan(test_df.iloc[test_index].connection) or math.isnan(qa_df.iloc[prediction].connection):
        continue

    # если в тесте и трейне совпала связка по посчитанным индексам из рейтинга, модели плюс очко
    if int(test_df.iloc[test_index].connection) == int(qa_df.iloc[prediction].connection):
        score += 1

In [103]:
"диппавлов + TF-IDF: " + str(score / len(rating))

'диппавлов + TF-IDF: 0.5347826086956522'

# БЕЗ ПРЕПРОЦЕССИНГА + TF-IDF

In [104]:
test_df_done = []
qa_df_done = []

for entry in tqdm(test_df['text']):
    test_df_done.append(str(entry))
for entry in tqdm(qa_df['text']):
    qa_df_done.append(str(entry))

both_done = qa_df_done + test_df_done

100%|██████████| 1652/1652 [00:00<00:00, 399365.43it/s]


In [105]:
vectorizer = TfidfVectorizer(both_done)
fit_vec = vectorizer.fit_transform(both_done)
X_train = vectorizer.transform(qa_df_done)
print(X_train.shape)
X_test = vectorizer.transform(test_df_done)
print(X_test.shape)

(1652, 14972)
(690, 14972)


In [106]:
# осталось только посчитать точность работы алгоритма

rating = X_train.dot(X_test.T).argmax(axis=0)
# в следующей ячейке у меня не получилось сделать проверку на nan через pandas,
# поэтому будет массив нампай
rating = np.array(rating)[0]
rating_en = enumerate(rating)

In [107]:
score = 0
# сравним связки
for test_index, prediction in rating_en:
    # проверка на NaN
    if math.isnan(test_df.iloc[test_index].connection) or math.isnan(qa_df.iloc[prediction].connection):
        continue

    # если в тесте и трейне совпала связка по посчитанным индексам из рейтинга, модели плюс очко
    if int(test_df.iloc[test_index].connection) == int(qa_df.iloc[prediction].connection):
        score += 1

In [108]:
"без проц + TF-IDF: " + str(score / len(rating))

'без проц + TF-IDF: 0.5478260869565217'

# bm25 из прошлой работы

### словарь обратн. инд.

In [ ]:
def dict_inversed(x_train):
    x_train = x_train.toarray()
    # это будет словарь вида dict('feature_name':[sum,  ind_1,ind_2, … ind_n,feature_index])
    output_dict = {}
    # маплю, получаю список названий фичей
    f_names = vectorizer.get_feature_names()
    for f_index, f_name in enumerate(f_names):
        # сумма единичек по горизонтали по фичам 
        output_dict[f_name] = [int(sum(x_train[:, f_index]))]
        for index, d_index in enumerate(x_train[:, f_index].tolist()):
            # проход по вертикали - по каждому документу. если встречается, то записываем
            if d_index != 0:
                output_dict[f_name].append(d_index)
        # последний параметр в списке для ключа-фичи - индекс фичи
        output_dict[f_name].append(f_index)
    return output_dict

# NATASHA + BM25 

# простите, не успеваю, не буду запускать, тогда совсем не успею собрать никакой проект

### __Задача 3__:    
Улучшить правила в natasha. Написать правила, которые ловят даты в следующих примерах и пересчитать статистику из Задачи 2:
- Уехал 8-9 ноября в Сочи
- Уезжаю 5 числа                           
- 20го сентября заболел

Пример можно посмотреть тут: https://github.com/natasha/yargy

In [111]:
# спасибо за помощь в реализации правил статье про разработку правил в ярги от Лаборатории Александра Кукушкина.
from yargy import rule, and_, or_, Parser
from yargy.predicates import gte, lte, caseless, normalized, dictionary

DAY = and_(
    gte(1),
    lte(31)
)
MONTH = and_(
    gte(1),
    lte(12)
)

MONTHS = {
    'январь',
    'февраль',
    'март',
    'апрель',
    'мая',
    'июнь',
    'июль',
    'август',
    'сентябрь',
    'октябрь',
    'ноябрь',
    'декабрь'
}

MONTH_NAME = dictionary(MONTHS)
DATE_WORDS = or_(
    rule(normalized('число')),
    rule(caseless('числа')),
    rule(caseless('го'))
)
DATE = or_(
    rule(
        DAY,
        DATE_WORDS.optional(),
        MONTH_NAME
    ),
    rule(
        DAY,
        DATE_WORDS.optional(),
        MONTH_NAME.optional()
    ),
    #Уехал 8-9 ноября в Сочи
    rule(
        DAY,
        "-",
        DAY,
        MONTH_NAME,
    )
)
parser = Parser(DATE)
text = '''
Уехал 8-9 ноября в Сочи
Уезжаю 5 числа
20го сентября заболел'''
for match in parser.findall(text):
    print(match.span, [_.value for _ in match.tokens])

[7, 17) ['8', '-', '9', 'ноября']
[32, 39) ['5', 'числа']
[40, 53) ['20', 'го', 'сентября']


Правила улучшены, простите, что не пересчитываю статистику, мне нужно успеть доделать 4 задание, чтобы собрать проект.